In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv(r'./Data/train.csv')
test=pd.read_csv(r'./Data/test.csv')

In [3]:
train_new=train.drop('id',axis=1)
Y_new=train_new.target
train_new.drop('target',1,inplace=True)
train_new_scaled=pd.DataFrame(StandardScaler().fit_transform(train_new),columns=train_new.columns)

In [4]:
xgc_params={'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 250}
xgc=xgb.XGBClassifier(**xgc_params)

In [5]:
rfc_params={'max_depth': 3, 'n_estimators': 400}
rfc=RandomForestClassifier(**rfc_params)

In [6]:
lgc_params={'C': 0.1, 'penalty': 'l1'}
lgc=LogisticRegression()

In [7]:
lgc.fit(train_new_scaled,Y_new)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [8]:
svc_params={'C': 0.5, 'degree': 1, 'kernel': 'poly', 'probability': True}
svc=SVC(**svc_params)

In [9]:
# vfc=VotingClassifier(estimators=[('rfc',rfc),('xgc',xgc),('svc',svc),('lgc',lgc)],voting='soft',n_jobs=4)

# vfc.fit(train_new_scaled,Y_new)

### Note 

I tried out other classifiers for the final stacked layer and ran a grid search for the best parameters but one with the highest accuracy is Logistic Regression.Makes one wonder doesn't it;)


In [10]:
id=test.id
test.drop('id',1,inplace=True)

In [11]:
test=pd.DataFrame(StandardScaler().fit_transform(test),columns=test.columns)

In [12]:
data_train=train_new.values
data_test=test.values
target=Y_new.values

In [13]:
kf=KFold(n_splits=5)

In [14]:
train_new.shape

(250, 300)

In [15]:
def get_output(clf,data_train,data_test,target):
    tr_out=np.zeros((train_new.shape[0],))
    ts_out=np.zeros((5,test.shape[0]))
    x=np.zeros((train_new.shape[0],))
    for i,(train_index,test_index) in enumerate(kf.split(data_train)):
        tr=data_train[train_index]
        ts=data_train[test_index]
        ty=target[train_index]
        
        clf.fit(tr,ty)
        tr_out[test_index]=pd.DataFrame(clf.predict_proba(ts))[1]
        ts_out[i]=pd.DataFrame(clf.predict_proba(data_test))[1]
    x=np.mean(ts_out,axis=0)
    return tr_out,x

In [16]:
x_xgc,y_xgc=get_output(xgc,data_train,data_test,target)
x_rfc,y_rfc=get_output(rfc,data_train,data_test,target)
x_lgc,y_lgc=get_output(lgc,data_train,data_test,target)
x_svc,y_svc=get_output(svc,data_train,data_test,target)

In [17]:
new_something=pd.DataFrame({'RandomForest':x_rfc,'SVC':x_svc,'Logistic':x_lgc,'XG':x_xgc})
new_something.shape

(250, 4)

In [18]:
new_test=pd.DataFrame({'RandomForest':y_rfc,'SVC':y_svc,'Logistic':y_lgc,'XG':y_xgc})

new_test.shape

(19750, 4)

In [19]:
Y_new.shape

(250,)

In [20]:
lgc_2=LogisticRegression(C=0.1)

In [21]:
lgc_2.fit(new_something,Y_new)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [22]:
preds=lgc.predict_proba(test)

In [23]:
s=pd.DataFrame(preds)

In [24]:
s['id']=id

In [25]:
s['target']=s[1]

In [26]:
s.drop(columns=[0,1],inplace=True)

In [27]:
s.to_csv("new2.csv",index=False)

In [28]:
s

,id,target
0,250,0.575607
1,251,0.034186
2,252,0.515224
3,253,0.999553
4,254,0.438289
5,255,0.411168
6,256,0.019094
7,257,0.919653
8,258,0.998491
9,259,0.117956
